In [16]:
# ! pip install selenium

You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import time
import openpyxl
from openpyxl.styles import PatternFill, Color
from datetime import datetime
from selenium import webdriver

In [2]:
# 브라우저 없이 selenium 사용 
webdriver_options = webdriver.ChromeOptions()
webdriver_options .add_argument('headless')

# selenium에서 사용할 웹 드라이버 절대 경로 정보
chromedriver = 'C:/Users/이찬희/python2021/chromedriver.exe'

# 해당 페이지의 상태를 가져온다. 
locate= '#stock-status > div.text-danger.stock-status-text'

# 상품의 고유번호을 가져온다. 
n_locate = '#product-specs-list > li:nth-child(4) > span'

# 엑셀의 row를 위한 변수
count = 1

In [3]:
# url을 담은 txt 파일 읽어오기 
def open_url():
    start = time.time()
    with open('C:/Users/이찬희/python2021/url.txt', 'rt', encoding='UTF8') as file:
        content = list()

        while True:
            sentence = file.readline()

            if sentence:
                content.append(sentence)
            else:
                break
        print("time :", time.time() - start)
        return (content)

In [4]:
open_url()

time : 0.0019922256469726562


['https://kr.iherb.com/pr/California-Gold-Nutrition-Omega-800-Pharmaceutical-Grade-Fish-Oil-80-EPA-DHA-Triglyceride-Form-1000-mg-90-Fish-Gelatin-Softgels/85180\n',
 'https://kr.iherb.com/pr/California-Gold-Nutrition-Omega-800-Pharmaceutical-Grade-Fish-Oil-80-EPA-DHA-1-000-mg-30-Fish-Gelatin-Softgels/82845\n',
 'https://kr.iherb.com/pr/California-Gold-Nutrition-LactoBif-Probiotics-30-Billion-CFU-60-Veggie-Capsules/64009?rec=iherbtest-home\n',
 'https://kr.iherb.com/pr/PEAKfresh-USA-Produce-Bags-with-Twist-Ties-Reusable-10-Bags/44160\n',
 'https://kr.iherb.com/pr/California-Gold-Nutrition-Gold-C-Powder-Vitamin-C-1-000-mg-8-81-oz-250-g/59561\n',
 'https://kr.iherb.com/pr/Kosette-Sanitizer-Wipes-Alcohol-Free-20-Wipes/103742']

In [11]:
def get_driver(): 
    start = time.time()
    content = open_url()
    # selenum의 webdriver에 앞서 설치한 chromedirver를 연동한다.
    driver = [webdriver.Chrome(chromedriver, options = webdriver_options ) for _ in content]
    
    # driver로 특정 페이지를 연다.
    for i in  range (len(content)):
        driver[i].get(content[i])
    print("time :", time.time() - start)
    return (driver)

In [12]:
get_driver()

time : 0.0
time : 33.44915175437927


[<selenium.webdriver.chrome.webdriver.WebDriver (session="35b618bdd4de8dd8425abee8f2916c99")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="c27f016468a8edae0046591133b5dddb")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="883915751d2d054c8ab7907d4f581d99")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="b6acd4280edea64269232f1fe04bb798")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="acf277559c1aea72cfdce8ffa84e2ab4")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="5b36719a2f210a3956255c4492b8ae79")>]

In [10]:
#상품의 고유 번호를 가져온다.   
def get_name(): 
    start = time.time()
    driver_list = get_driver()
    name_list = list()
    for i in driver_list:
        name_list.append(i.find_element_by_css_selector(n_locate).text)
    
    print("time :", time.time() - start)
    return (name_list)

In [13]:
get_name()

time : 0.0009984970092773438
time : 40.62613534927368
time : 42.413355112075806


['898220012664',
 'CGN-01251',
 'CGN-00965',
 '892453002003',
 'CGN-00935',
 'IMI-21029']

In [14]:
#상품의 재고 상황을 가져온다.
def scrapping():
    start = time.time()
    status_list = list()
    driver_list = get_driver()
     
    for i in driver_list:
        try:
            status_list.append(i.find_element_by_css_selector(locate).text)

        except:
                status_list.append('재고 있음')
    print("time :", time.time() - start)
    return (status_list)

In [15]:
scrapping()

time : 0.0009980201721191406
time : 51.99909448623657
time : 53.05925965309143


['품절', '품절', '재고 있음', '재고 있음', '재고 있음', '재고 있음']

In [21]:
# 워크북(엑셀파일)을 새로 만듭니다.
wb = openpyxl.Workbook()

sheet1 = wb['Sheet']

# 고유번호 값을 가져온다. 
name_list = get_name()
content = open_url()

# 헤더를 추가한다. 
sheet1.append(["날짜", "시간"])
sheet1.column_dimensions['A'].width = 15

# 고유번호로 상품의 이름을 보여주고, 하이퍼링크를 통해 누르면 상품 사이트와 연결되도록 한다. 
for i in range (len(name_list)):
    sheet1.cell(row=1, column=i+3).value = '=HYPERLINK("{}", "{}")'.format(content[i], name_list[i])
    sheet1.cell(row=1, column=i+3).style = "Hyperlink" # 하이퍼링크처럼 파란색으로 보여주는 코드

wb.save('재고확인.xlsx')

In [20]:
wb = openpyxl.load_workbook('재고확인.xlsx')
status_list = list()

while True:  
    try:
        start = time.time()
        #시간 설정 
        now = datetime.now()
        date = "%s년 %s월 %s일" %(now.year, now.month, now.day)
        hour = "%s시 %s분" %(now.hour, now.minute)

        # 1분마다 
        # scrapping 함수를 호출 
        print("scrapping ... ")
        status_list = scrapping()

        sheet1 = wb.active
        sheet1.append([date, hour])

        # 각 셀에 상태 저장 
        for i in range (len(status_list)):
            if (status_list[i] == '품절'):
                sheet1.cell(row=count+1, column=i+3).value = status_list[i]
                sheet1.cell(row=count+1, column=i+3).fill = PatternFill(start_color='00FF0000', end_color='00FF0000', fill_type='solid')
            elif (status_list[i] == '재고 있음'):
                sheet1.cell(row=count+1, column=i+3).value = status_list[i]
                sheet1.cell(row=count+1, column=i+3).fill = PatternFill(start_color='0099CC00', end_color='0099CC00', fill_type='solid')

        #1분마다 정보를 갱신한다. 
        time.sleep(10) 
        count += 1 # 엑셀의 row를 위한 변수 
        
        wb.save('재고확인.xlsx')
        print("time :", time.time() - start)
    except:
        print("Error")
        break
    

    # 시트별로 나누는 거 말고 컬럼별로 해서 하나의 시트에서 만드는 걸로 생각해보기
    # 함수화 하는 거 생각해보기 
    # 변수 초기화 한 번에 하기 
    # URL 파일 주기적으로 확인하기 

scrapping ... 
time : 115.03667759895325
scrapping ... 
time : 54.69095325469971
scrapping ... 
Error


In [ ]:
get_name()